In [3]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.invoke("how many letters in the word educa?")
response

AIMessage(content='There are 6 letters in the word "educa".')

### Parse email dataset

In [ ]:
import pandas as pd  
# you can download from https://www.kaggle.com/datasets/wcukierski/enron-email-dataset/
df = pd.read_csv('./emails.csv',nrows=10)

In [ ]:
from apihelper.parsers import email_parser
with open(f"../../data/emails.txt","w") as fd:
    for item in df['message']:
        parsed_item,isok = email_parser(item)
        if isok:
            fd.write(f"{parsed_item}\n\n")

### Download and parse chat dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("FunDialogues/customer-service-robot-support", split="train")

In [ ]:
from apihelper.parsers import chat_parser
with open(f"../../data/chats.txt","w") as fd:
    for item in dataset['dialogue'][:10]:
        parsed_item,isok = chat_parser(item)
        if isok:
            fd.write(f"{parsed_item}\n\n")

### Quadrant test

In [5]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.docstore.document import Document


In [6]:
loader = TextLoader("../../data/emails.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1780, which is longer than the specified 500


In [7]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="../../data/quadrant_emails",
    collection_name="emails",
)

In [ ]:
loader = TextLoader("../../data/chats.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [ ]:
qdrant_chats = Qdrant.from_documents(
    docs,
    embeddings,
    path="../../data/quadrant",
    collection_name="chats",
)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search_with_score(query)